## Preparing data

In [1]:
import xarray as xr
import geopandas as gpd
from dask.distributed import Client, LocalCluster
from datetime import datetime, timedelta
from functools import partial
from PyStemmusScope import variable_conversion as vc
from rasterio.warp import reproject, Resampling
import numpy as np
import glob
import dask.array as da
import pandas as pd
from dask_jobqueue import SLURMCluster

In [2]:
year = 2019

parent_in_path = f"/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data"
parent_in_path1 = f"/projects/0/einf2480/era5land2000_2009_2020"

data_paths = {
            "era5land": f"{parent_in_path1}/{2008}/era5-land_*.nc",
            "lai": f"{parent_in_path}/{year}global/lai_v2/*.nc",
            "ssm": f"{parent_in_path}/{year}global/ssm/GlobalGSSM11km{year}_20240214withbandname.tif",
            "co2": f"{parent_in_path}/{2015}global/co2/CAMS_CO2_2003-2020.nc",
            "landcover": f"{parent_in_path}/landcover/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2013-v2.0.7cds.nc",
            "vcmax": f"{parent_in_path}/Vcmax/TROPOMI_Vmax_Tg_mean.tif",
            "canopyheight": f"{parent_in_path}/canopy_height/canopy_height_11kmGlobal20240215.tif",
            }

parent_out_path = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/testSarah"

In [3]:
def era5_preprocess(ds):    
    if 'valid_time' in ds.dims:
        ds = ds.rename({'valid_time': 'time'})
        
    ds = ds.assign_coords(
        latitude=ds.latitude.astype('float32'),
        longitude=ds.longitude.astype('float32')
    )
    ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))
    return ds

def fix_coords(ds):
    if 'band' in ds.dims:
        ds = ds.rename_dims({'band': 'time'})
        ds = ds.rename_vars({'band': 'time'})

    if 'x' in ds.dims and 'y' in ds.dims:
        ds = ds.rename_dims({'x': 'longitude', 'y': 'latitude'})
        ds = ds.rename_vars({'x': 'longitude', 'y': 'latitude'})
        
    elif 'lon' in ds.dims and 'lat' in ds.dims:
        ds = ds.rename_dims({'lon': 'longitude', 'lat': 'latitude'})
        ds = ds.rename_vars({'lon': 'longitude', 'lat': 'latitude'})
    return ds

In [4]:
cluster = SLURMCluster(
    name='dask-worker',
    cores=16,
    processes=16,
    queue='fat',
    memory='120GiB',
    local_directory='$TMPDIR',
    walltime='3:00:00'
)
cluster.scale(jobs=4)
client = Client(cluster)
client

/home/qiahan/.conda/envs/mamba/envs/emulator/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40247 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/40247/status,
Dashboard: /proxy/40247/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://145.136.63.37:45149,Workers: 0
Dashboard: /proxy/40247/status,Total threads: 0
Started: Just now,Total memory: 0 B


## ERA5LAND

In [5]:
%%time
era5land = xr.open_mfdataset(data_paths['era5land'], preprocess=era5_preprocess, chunks={'longitude': 250, 'latitude': 250})
# era5land = era5land.chunk({'time': 750})
era5land = era5land.sortby(['longitude', 'latitude'])
era5land = era5land.chunk(
    time=750, 
    longitude=250, 
    latitude=250
)
# # svae to zarr
# out_path = f"{parent_in_path}/{year}global/era5land/{'era5land'}_{year}.zarr"
# era5land.to_zarr(out_path, mode='w')

CPU times: user 4.36 s, sys: 669 ms, total: 5.03 s
Wall time: 24.4 s


In [6]:
target = era5land['sp'].isel(time=0, drop=True)
target = target.rio.write_crs('EPSG:4326')

## CO2

In [6]:
%%time
## CO2
co2 = xr.open_dataset(data_paths['co2']).sel(time=str(year))
co2 = co2.assign_coords(longitude=(((co2.longitude + 180) % 360) - 180))     
co2 = co2.sortby(['longitude', 'latitude'])
co2 = vc.co2_mass_fraction_to_kg_per_m3(co2['co2'])*1e6

CPU times: user 4.94 s, sys: 783 ms, total: 5.73 s
Wall time: 5.99 s


In [7]:
# co2 = co2.sel(time=slice('2014-01-01T00:00:00.000000000','2014-01-31T03:00:00.000000000'))

In [8]:
%%time
co2 = co2.rio.write_crs('EPSG:4326')
co2.rio.write_nodata(co2.rio.nodata, inplace=True)
co2_reprojected = co2.rio.reproject_match(target, resampling=Resampling.average, nodata=np.nan)
co2_reprojected = co2_reprojected.assign_coords({
    "x": co2_reprojected.x,
    "y": co2_reprojected.y,
    'time':('time',co2_reprojected.time.values + np.timedelta64(3,'h'),co2_reprojected.time.attrs) # convert from starting time to ending time, same as ERA5Land
})
co2_reprojected = co2_reprojected.rename({'x':'longitude','y':'latitude'})


CPU times: user 11min 34s, sys: 18.5 s, total: 11min 53s
Wall time: 11min 55s


In [9]:
%%time
co2_reprojected = co2_reprojected.chunk(
    time=-1, 
    longitude=250, 
    latitude=250
)
co2_reprojected = co2_reprojected.to_dataset(name='co2')


CPU times: user 57 s, sys: 3.86 ms, total: 57 s
Wall time: 57.2 s


In [10]:
co2_reprojected

<xarray.Dataset>
Dimensions:      (longitude: 3600, latitude: 1801, time: 2928)
Coordinates:
  * longitude    (longitude) float32 -180.0 -179.9 -179.8 ... 179.7 179.8 179.9
  * latitude     (latitude) float32 -90.0 -89.9 -89.8 -89.7 ... 89.8 89.9 90.0
  * time         (time) datetime64[ns] 2020-01-01T03:00:00 ... 2021-01-01
    spatial_ref  int64 0
Data variables:
    co2          (time, latitude, longitude) float32 dask.array<chunksize=(2928, 250, 250), meta=np.ndarray>

In [11]:
%%time
# svae to zarr
out_path = f"{parent_in_path}/{year}global/co2/{'co2'}_{year}.zarr"
co2_reprojected.to_zarr(out_path, mode='w')

CPU times: user 1min 52s, sys: 46.5 s, total: 2min 39s
Wall time: 11.2 s


In [12]:
f"{parent_in_path}/{year}global/co2/{'co2'}_{year}.zarr"

'/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2020global/co2/co2_2020.zarr'

## LAI

In [7]:
LAI_DIR_before = (f"{parent_in_path}/{year-1}global/lai_v2/")
LAI_DIR_after = (f"{parent_in_path}/{year+1}global/lai_v2/")
LAI_DIR = (f"{parent_in_path}/{year}global/lai_v2/")
lai_paths_year = sorted(
    glob.glob(f'{LAI_DIR}/c_gls_LAI*_GLOBE_*.nc') #different from LAI_V1
)
lai_paths_before = sorted(
    glob.glob(f'{LAI_DIR_before}/c_gls_LAI*_GLOBE_*.nc')
)[-1]
lai_paths_after = sorted(
    glob.glob(f'{LAI_DIR_after}/c_gls_LAI*_GLOBE_*.nc')
)[6]
lai_paths = []
lai_paths.append(lai_paths_before)
lai_paths.extend(lai_paths_year)
lai_paths.append(lai_paths_after)

In [8]:
lai = xr.open_mfdataset(lai_paths, chunks={})
lai = lai['LAI']  # keep only one variable
lai.rio.write_nodata(np.nan, inplace=True)
lai = lai.rio.write_crs('WGS84')
lai = lai.rename(
    lon='longitude',
    lat='latitude',
)

In [9]:
lai

<xarray.DataArray 'LAI' (time: 38, latitude: 15680, longitude: 40320)>
dask.array<concatenate, shape=(38, 15680, 40320), dtype=float32, chunksize=(1, 15680, 40320), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float64 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * latitude   (latitude) float64 80.0 79.99 79.98 ... -59.97 -59.98 -59.99
  * time       (time) datetime64[ns] 2018-12-31 2019-01-10 ... 2020-01-10
    crs        int64 0
Attributes:
    grid_mapping:   crs
    valid_range:    [  0 210]
    long_name:      Leaf Area Index 1km
    standard_name:  leaf_area_index
    units:          
    _FillValue:     nan

In [10]:
template = xr.DataArray(
    data=da.zeros(
        (len(lai.time), len(target.latitude), len(target.longitude)),
        chunks=(1, -1, -1),
    ),
    dims=('time', 'latitude', 'longitude'),
    coords={
        'time': lai.time, 
        'latitude': target.latitude, 
        'longitude': target.longitude,
    },
)
def reproject(source):
    # import here, otherwise not seen by workers
    import rioxarray
    from rasterio.warp import reproject, Resampling
    reprojected = source.rio.reproject_match(
        target,
        nodata=np.nan,
        resampling=Resampling.average
    )
    reprojected = reprojected.drop_vars('crs')
    return reprojected.rename(x='longitude', y='latitude')

lai_reprojected = xr.map_blocks(
    reproject,
    lai,
    template=template,
)

In [11]:
%%time
lai_reprojected = lai_reprojected.chunk(
    time=-1, 
    longitude=250, 
    latitude=250
)
lai_reprojected = lai_reprojected.to_dataset(name='LAI')
# svae to zarr
out_path = f"{parent_in_path}/{year}global/lai_v2/{'lai_v2'}_{year}correct.zarr"
lai_reprojected.to_zarr(out_path, mode='w')

CPU times: user 7.41 s, sys: 426 ms, total: 7.83 s
Wall time: 29.5 s


In [12]:
print(lai_reprojected['latitude'].encoding)
print(lai_reprojected['longitude'].encoding)


{}
{}


In [13]:
print(lai_reprojected['latitude'].dtype)  # 确认为 float64
print(lai_reprojected['longitude'].dtype)  # 确认为 float64

float64
float64


In [14]:
client.shutdown()

## SSM

In [7]:
%time
ssm = xr.open_dataset(data_paths['ssm'])
ssm = ssm.rio.write_crs('WGS84')
ssm = fix_coords(ssm)
ssm = ssm['band_data']
# ssm['time'] = pd.to_datetime(ssm.time-1, unit='D', origin=str(data_paths['ssm'].split('/')[-1][14:18]))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 8.34 µs


In [8]:
ssm

<xarray.DataArray 'band_data' (time: 357, latitude: 1480, longitude: 3600)>
[1902096000 values with dtype=float32]
Coordinates:
  * time         (time) int64 1 2 3 4 5 6 7 8 ... 351 352 353 354 355 356 357
  * longitude    (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * latitude     (latitude) float64 87.95 87.85 87.75 ... -59.75 -59.85 -59.95
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    long_name:      ('band_2016_01_01_classification', 'band_2016_01_02_class...

In [9]:
import xarray as xr
import re
import pandas as pd

# 假设 ds 是你的 xarray Dataset
variable_names = list(ssm.long_name)  # 获取所有变量名

# 使用正则表达式提取日期部分
date_strings = [re.search(r'band_(\d{4}_\d{2}_\d{2})_classification', var).group(1) for var in variable_names]

# 转换为 pandas datetime 格式
dates = pd.to_datetime(date_strings, format='%Y_%m_%d')

# 赋值给 Dataset 的 time 维度
ssm = ssm.assign_coords(time=("time", dates))


In [10]:
ssm.rio.write_nodata(np.nan, inplace=True)
SSM0 = ssm
SSM0 = SSM0.rio.write_crs('EPSG:4326')
target = target.rio.write_crs('EPSG:4326')
ssm.rio.write_nodata(SSM0.rio.nodata, inplace=True)
ssm_reprojected = SSM0.rio.reproject_match(target, resampling=Resampling.average, nodata=np.nan) #
ssm_reprojected = ssm_reprojected.rename(
    x='longitude',
    y='latitude',
)
# ssm_reprojected.isnull().sum()

In [11]:
%%time
ssm_reprojected = ssm_reprojected.chunk(
    time=-1, 
    longitude=250, 
    latitude=250
)
ssm_reprojected = ssm_reprojected.to_dataset(name='SSM')

CPU times: user 6.95 s, sys: 3.99 ms, total: 6.95 s
Wall time: 6.98 s


In [12]:
%%time
# svae to zarr
out_path = f"{parent_in_path}/{year}global/ssm/{'ssm_global'}_{year}correct.zarr"
ssm_reprojected.to_zarr(out_path, mode='w')

CPU times: user 18.9 s, sys: 6.88 s, total: 25.8 s
Wall time: 2.51 s


## landcover

In [30]:
landcover = xr.open_mfdataset(data_paths['landcover'])["lccs_class"]
landcover = landcover.rio.write_crs('WGS84')
landcover = landcover.rename(
    lon='longitude',
    lat='latitude',
)

In [31]:
landcover = landcover.rio.write_crs('EPSG:4326')
landcover.rio.write_nodata(landcover.rio.nodata, inplace=True)
landcover_reprojected = landcover.rio.reproject_match(target, resampling=Resampling.mode)
landcover_reprojected = landcover_reprojected.assign_coords({
    "x": landcover_reprojected.x,
    "y": landcover_reprojected.y,
})
# fix naming of coordinates
landcover_reprojected = landcover_reprojected.rename(
    x='longitude',
    y='latitude'
)

In [32]:
landcover_reprojected = landcover_reprojected.to_dataset(name='lccs_class')
landcover_reprojected = landcover_reprojected.chunk(
    longitude=250, 
    latitude=250
)

In [33]:
%%time
# svae to zarr
out_path = f"{parent_in_path}/landcover/{'landcover'}.zarr"
landcover_reprojected.to_zarr(out_path, mode='w')

CPU times: user 55.3 ms, sys: 57.9 ms, total: 113 ms
Wall time: 120 ms


## canopy height (hc)

In [34]:
hc = xr.open_dataset(data_paths['canopyheight']).band_data
# hc = hc.squeeze('band')  # drop band dimension
hc = hc.rio.write_crs('WGS84')
hc = hc.rename(
    x='longitude',
    y='latitude',
)
hc = hc.sortby(["longitude", "latitude"])

In [35]:
import numpy as np
hc.rio.write_nodata(np.nan, inplace=True)
hc = hc.rio.write_crs('EPSG:4326')
target = target.rio.write_crs('EPSG:4326')
hc_reprojected = hc.rio.reproject_match(target, resampling=Resampling.average, nodata=np.nan)
hc_reprojected = hc_reprojected.assign_coords({
    "x": hc_reprojected.x,
    "y": hc_reprojected.y,
})
# fix naming of coordinates
hc_reprojected = hc_reprojected.rename(
    x='longitude',
    y='latitude'
)

In [36]:
hc_reprojected = hc_reprojected.to_dataset(name='hc')
hc_reprojected = hc_reprojected.chunk(
    longitude=250, 
    latitude=250
)
# svae to zarr
out_path = f"{parent_in_path}/canopy_height/{'hc_global'}.zarr"
hc_reprojected.to_zarr(out_path, mode='w')

## Vcmax

In [37]:
vcmax = xr.open_dataset(data_paths['vcmax']).band_data
vcmax.rio.write_crs("EPSG:4326", inplace=True)
vcmax.rio.write_nodata(np.nan, inplace=True)

<xarray.DataArray 'band_data' (band: 1, y: 360, x: 720)>
[259200 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * y            (y) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        185.50468444824
    STATISTICS_MEAN:           32.584587871174
    STATISTICS_MINIMUM:        0.44747722148895
    STATISTICS_STDDEV:         30.55013573823
    STATISTICS_VALID_PERCENT:  20.61
    _FillValue:                nan

In [38]:
vcmax_fillna = vcmax.rio.interpolate_na(method='nearest') #interpolation method see: scipy.interpolate.griddata
vcmax_fillna = vcmax_fillna.rio.write_crs('WGS84')
vcmax_fillna = vcmax_fillna.rename(
    x='longitude',
    y='latitude',
)

In [39]:
vcmax_fillna_reprojected = vcmax_fillna.rio.reproject_match(target, resampling=Resampling.average, nodata=np.nan)
vcmax_fillna_reprojected = vcmax_fillna_reprojected.assign_coords({
    "x": vcmax_fillna_reprojected.x,
    "y": vcmax_fillna_reprojected.y,
})
# fix naming of coordinates
vcmax_fillna_reprojected = vcmax_fillna_reprojected.rename(
    x='longitude',
    y='latitude'
)

In [40]:
vcmax_fillna_reprojected = xr.where(target.notnull(), vcmax_fillna_reprojected, np.nan)
vcmax_fillna_reprojected_mask = vcmax_fillna_reprojected.where(vcmax_fillna_reprojected['latitude'] > -63, other=float('nan'))

In [41]:
vcmax_fillna_reprojected_mask = vcmax_fillna_reprojected_mask.to_dataset(name='vcmax')

In [43]:
%%time
vcmax_fillna_reprojected_mask = vcmax_fillna_reprojected_mask.chunk(
    longitude=250, 
    latitude=250
)
# svae to zarr
out_path = f"{parent_in_path}/Vcmax/{'vcmax_fillnan'}.zarr"
vcmax_fillna_reprojected_mask.to_zarr(out_path, mode='w')

CPU times: user 2min 18s, sys: 7min 41s, total: 9min 59s
Wall time: 8min 22s
